In [2]:
from transform import convert_images
from augment import augment_images
from split import split_dataset
from utils import input_folder, output_folder,augment_folder, final_data_folder
import os

In [3]:
convert_images(input_folder, output_folder, target_format=".jpeg")
augment_images(output_folder, augment_folder, target_size=(256, 256), target_format=".jpeg")

Output directory created: /home/anant/anaconda3/envs/class/project/updated_furnture_data
Output directory created: /home/anant/anaconda3/envs/class/project/augmented


In [4]:
split_dataset(augment_folder, final_data_folder)

Processed class: table, 48 train, 6 val, 6 test
Processed class: chair, 43 train, 5 val, 6 test
Processed class: bed, 48 train, 6 val, 6 test
Processed class: storage, 48 train, 6 val, 6 test
Processed class: sofa, 48 train, 6 val, 6 test


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from transform import transform
import time
import os
from utils import final_data_folder
import numpy as np
from PIL import Image
from torchinfo import summary

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image (optional, based on model input)
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

train_data = datasets.ImageFolder(root=os.path.join(final_data_folder, 'train'), transform=transform)
test_data = datasets.ImageFolder(root=os.path.join(final_data_folder, 'test'), transform=transform)
val_data = datasets.ImageFolder(root=os.path.join(final_data_folder, 'val'), transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

sample_image, label = train_data[0]

# Print out the type of the image
print(f"Type of the image before transformation: {type(sample_image)}")
print(f"Shape of the image before transformation: {np.array(sample_image).shape if isinstance(sample_image, Image.Image) else sample_image.shape}")

Type of the image before transformation: <class 'torch.Tensor'>
Shape of the image before transformation: torch.Size([3, 224, 224])


In [4]:
# Define function to create models
def create_model(model_name, num_classes=10):
    """
    Create and return a model based on the name and number of output classes.
    """
    if model_name == 'resnet':
        model = models.resnet50(weights=models.ResNet50_Weights)  # ResNet18 (can use ResNet50, ResNet101 as well)
        model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify final layer for custom classes
    
    elif model_name == 'vgg16':
        model = models.vgg16(weights=models.VGG16_Weights)  # VGG16
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)  # Modify final layer
    
    elif model_name == 'densenet':
        model = models.densenet201(weights=models.DenseNet201_Weights)  # DenseNet121
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)  # Modify final layer
    
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    # Freeze all layers except the last ones
    for param in model.parameters():
        param.requires_grad = False  # Freeze all layers
    
    # Unfreeze the last layer
    if model_name == 'resnet':
        model.fc.weight.requires_grad = True
    elif model_name == 'vgg16':
        model.classifier[6].weight.requires_grad = True
    elif model_name == 'inceptionv3':
        model.fc.weight.requires_grad = True
    elif model_name == 'densenet':
        model.classifier.weight.requires_grad = True

    return model

In [5]:
# Function to train a model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    """
    Train the given model on the dataset.
    """
    model = model.to(device)
    model.train()  # Set model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        end_time = time.time()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        epoch_time = end_time - start_time
        
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Time: {epoch_time:.2f}s")

In [6]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    """
    Evaluate the model on the test set.
    """
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
# device = 'cpu'
# Main function to train and evaluate models
def main():
    # model_names = ['resnet', 'vgg16', 'densenet']  # List of models to train
    model_names = ['densenet']
    num_classes = 5

    for model_name in model_names:
        print(f"\nTraining {model_name} model:")
        
        model = create_model(model_name, num_classes=num_classes)
        model = model.to(device)

        # Loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        trainable_params = [p for p in model.parameters() if p.requires_grad]
        if not trainable_params:
            raise ValueError("No trainable parameters found. Make sure final layers are unfrozen.")

        # Optimizer
        optimizer = optim.Adam(trainable_params, lr=0.001)
        # optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

        # Train the model
        train_model(model, train_loader, criterion, optimizer, num_epochs=30)

        # Evaluate the model
        evaluate_model(model, test_loader)

if __name__ == "__main__":
    main()


Training densenet model:


/home/anant/anaconda3/envs/class/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/30 - Loss: 1.4249, Accuracy: 41.20%, Time: 3.62s
Epoch 2/30 - Loss: 0.9215, Accuracy: 87.68%, Time: 3.32s
Epoch 3/30 - Loss: 0.5774, Accuracy: 96.48%, Time: 3.33s
Epoch 4/30 - Loss: 0.3995, Accuracy: 96.83%, Time: 3.33s
Epoch 5/30 - Loss: 0.3107, Accuracy: 98.24%, Time: 3.32s
Epoch 6/30 - Loss: 0.2590, Accuracy: 98.94%, Time: 3.34s
Epoch 7/30 - Loss: 0.2025, Accuracy: 98.94%, Time: 3.35s
Epoch 8/30 - Loss: 0.1663, Accuracy: 99.65%, Time: 3.34s
Epoch 9/30 - Loss: 0.1410, Accuracy: 99.30%, Time: 3.36s
Epoch 10/30 - Loss: 0.1280, Accuracy: 99.30%, Time: 3.34s
Epoch 11/30 - Loss: 0.1320, Accuracy: 100.00%, Time: 3.35s
Epoch 12/30 - Loss: 0.0939, Accuracy: 100.00%, Time: 3.35s
Epoch 13/30 - Loss: 0.0860, Accuracy: 100.00%, Time: 3.37s
Epoch 14/30 - Loss: 0.0759, Accuracy: 100.00%, Time: 3.38s
Epoch 15/30 - Loss: 0.0781, Accuracy: 100.00%, Time: 3.36s
Epoch 16/30 - Loss: 0.0816, Accuracy: 99.30%, Time: 3.37s
Epoch 17/30 - Loss: 0.0707, Accuracy: 100.00%, Time: 3.39s
Epoch 18/30 - Los